<a href="https://colab.research.google.com/github/SzymonFabian/Agg_PKB_streamlit/blob/main/tutorial_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/worldbank/OpenNightLights/blob/master/onl/tutorials/mod2_6_practical_exercise-image_visualization.ipynb)

# Practical exercise: image visualization (10 min)

When you’re working with maps and images, what is the first thing you want to do? Display it! In this exercise, we’re going to display nighttime lights for Nigeria in 1992 and 2013.

**Prerequisites:**
- Make sure you have Python, Jupyter notebooks, and geemap installed and are familiar with these packages
- If not, you'll want to review the earlier sections in this module.

**Our tasks in this exercise:**
1. Initialize a map object with geemap
2. Query DMSP-OLS data for 1992
3. Create (and adjust) the Nigeria 1992 nighttime lights layer
4. Change the map object's basemap
5. Visual inspection
6. Repeat steps 2 and 3 for DMSP-OLS 2013 data

## Initialize a map object with geemap

Later in this tutorial, we're going to show you how to import other geospatial data files, such as the boundaries to countries or sub-national regions, to help you analyze nighttime lights.

For now, we're going to focus our scene on Nigeria, but we'll do that simply by centering our map on the capital city of Abuja, which we can find at approximately: latitude: 9.0 and longitude: 7.4.

We'll set our map zoom factor to 6 to include the entire country in our view.

You should get in the habit of saving parameters as variables. This makes it easy to re-use your code for different values.

In [ ]:
try:
    import geemap, ee
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        print("package not found, installing w/ pip in Google Colab...")
        !pip install geemap
    else:
        print("package not found, installing w/ conda...")
        !conda install mamba -c conda-forge -y
        !mamba install geemap -c conda-forge -y
    import geemap, ee

In [5]:
try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

# set our initial map parameters for Abuja, Nigeria
center_lat = 9.0
center_lon = 7.4
zoomlevel=6

# initialize our map
myFirstMap = geemap.Map(center=[center_lat,center_lon], zoom=zoomlevel)

<div class="alert alert-warning">
Recall that to use GEE you need an account. The first time you execute this cell, you'll be prompted to enter your authentication code. Another browser window will open where you can select the Google account you have associated with your GEE account. Copy the authentication code provided and enter it in the input box provided in this notebook and hit enter and you should be ready to go. You should not need to do this again on this computer. </div>

In [6]:
# display our map
myFirstMap.addLayerControl()
myFirstMap

Map(center=[9.0, 7.4], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

<div class="alert alert-warning">
Once you've authenticated. Executing the previous cell (selecting `Run` or SHIFT + ENTER) should display a map, centered on the Nigerian capital city of Abuja.<br><br>
If you don't see a map: don't panic! But you've run into one of the challenges of interpreted open-source languages -- your Python packages and dependencies, including Jupyter notebook, and your computer's "back-end" software may not be in sync.</div>

First, make sure `geemap` and `jupyter` are installed properly as per our earlier tutorials referenced above. If you're using a virtual environment, make sure that's activated and geemap is installed within it!

If that doesnt resolve the issue, another fix is to make sure your Jupyter extensions enable `ipyleaflet` by running this code in command line.

Recall from an earlier tutorial that in Jupyter notebooks you can make system calls (such as you would in command line prompt) by adding a bang (exclamaition point) to your notebook code, like this:

In [ ]:
# !jupyter nbextension enable --py --sys-prefix ipyleaflet

Enabling notebook extension jupyter-leaflet/extension...
      - Validating: OK


You may have to close out and restart your notebook and even your Python session after this, but you should only have to do this once.

**A note on troubleshooting**
At this point, hopefully, you've got `geemap` set up and working.

If not, you'll want to troubleshoot possible issues and there are many forums for discussing common problems, such as <a href="https://stackoverflow.com/">stackoverflow.com</a>.

If you learn to do this, it will serve you well. Troubles can be frustrating, but "shooting" them requires problem-solving that helps you better understand what you're doing and become a better programmer and data scientist.


## Query DMSP-OLS data for 1992

### Get DMSP-OLS image ID for 1992

Now we want to query our nighttime lights. We're looking at the DMSP-OLS series. **Images** are what Google Earth Engine uses to describe raster files and **ImageCollections** are collections (like a time series) of images. We're looking for the DMSP-OLS Nighttime lights ImageCollection.

<a href="https://code.earthengine.google.com/">https://code.earthengine.google.com/</a>

It's easy to search for particular collections. For example, if you're using the GEE code editor, search for "DMSP" and it will prompt you for 2 collections: we want the nighttime lights.

![DMSP_search](https://github.com/worldbank/OpenNightLights/blob/master/onl/tutorials/img/mod1_7_fig2.png?raw=1)

Select the collection for "Nighttime lights" and in the window that appears, you'll see the exact collect ID (highlighed in the lower left). `NOAA/DMSP-OLS/NIGHTTIME_LIGHTS`

![DMSP_window](https://github.com/worldbank/OpenNightLights/blob/master/onl/tutorials/img/mod1_7_fig3.png?raw=1)

For quering this data via the API, this ImageCollection ID is what we are looking for: `NOAA/DMSP-OLS/NIGHTTIME_LIGHTS`

That full collection is a lot of data; however. And all we really need is the image for 1992. Since the DMSP-OLS nighttime lights are available annually, we're looking for a single Image.

Recall that the DMSP is composed of six satellites spanning 1992 to 2013. The satellite designated "F10" is what provided images for 1992.

**HINT** from {doc}`mod1_2_introduction_to_nighttime_light_data`: <a href="https://ngdc.noaa.gov/eog/dmsp/downloadV4composites.html" class="alert-link">NOAA`s
National Center for Environmental Information</a> gives the mapping of years and DMSP satellites.

Putting it together, the full file ID we need to query this image from GEE is: `NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F101992`

Again, let's get in the habit of setting variables so we avoid repetition (including repeated typos!)

In [ ]:
dmsp92id = "NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F101992"

## Create (and adjust) the Nigeria 1992 nighttime lights layer

Now that we know what Image we're looking for, we can query it via the Python API and add it as a layer to another map. You should be able to just add this layer to your existing map object, but the implementation in Colab is a little tricky so for we'll just create new maps for simplicity.

In [ ]:
# create an ee object for our 1992 image
# note that for DMSP, there is only one band, so we dont need to worry about selecting a band.
dmsp92 = ee.Image(dmsp92id)

# initialize another map add this image as a layer to our map object
# and call the layer: "DMSP NTL 1992"
Map2 = geemap.Map(center=[center_lat,center_lon], zoom=zoomlevel)
Map2.addLayer(dmsp92, name='DMSP NTL 1992')

Map2.addLayerControl()
Map2

Map(center=[9.0, 7.4], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

Voila! We have a nighttime layer from the 1992 DMSP-OLS composite.

### Changing opacity
You may notice it's quite dark; however. You can always toggle the layer off, but if you want to visualize the nighttime lights over the basemap, you'll want to change the opacity of your nighttime lights layer. Fortunately, this is very easy for us to do.

Our `.addLayer` function allows for other visual parameters, like `opacity`. Let's give this layer an opacity of 75%:

In [ ]:
Map3 = geemap.Map(center=[center_lat,center_lon], zoom=zoomlevel)
Map3.addLayer(dmsp92, name='DMSP NTL 1992', opacity=0.75)

Map3.addLayerControl()
Map3

Map(center=[9.0, 7.4], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

## Creating a mask
Another important step to "clean" your image will be to create a mask that filters out zero or negative values, which can happen after preprocessing for noisy and low-light pixels.

This can be done when adding (or updating) a layer. The ee Image object we created, `dmsp92`, has a built-in method called `.mask()` and when we call that and pass the Image itself as an argument, we get the mask.

<div class="alert alert-success">
<a href="https://developers.google.com/earth-engine/tutorials/tutorial_api_05#masking">This documentation</a> gives  more info on the GEE API .mask() call and we'll get into more detail on these data processing steps later.</div>

This time, let's change the name so that we create a new layer. Then we'll have a masked and non-masked 1992 layer:

In [ ]:
Map4 = geemap.Map(center=[center_lat,center_lon], zoom=zoomlevel)
Map4.addLayer(dmsp92.mask(dmsp92), name='DMSP NTL 1992 masked', opacity=0.75)

Map4.addLayerControl()
Map4

Map(center=[9.0, 7.4], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

## Change the basemap

Go to the cell with our map object and re-run the cell, updating it. You should now have two layers (intial 1992 and masked 1992) and you can toggle between them. You can imagine that the masked layer makes it easier to inspect the underlying basemap layer.

The default basemap is Open Street Maps (OSM). But if you want to analyze nighttime lights according to land build-up as seen in daylight images (such as from LANDSAT), you can change the basemap (if you're more advanced you can search GEE for your own layers of course).

There are a few dozen options to choose from for geemap basemaps. While there's not documentation yet, you can see the options in the <a href="https://github.com/giswqs/geemap/blob/master/geemap/basemaps.py">source code itself.</a>. Some of these have also been added to `ipyleaflet`'s library.

Navigate to that source code link and review the options. Let's choose the default "SATELLITE" basemap which appears to be of the Google maps daytime satellite view.

In [ ]:
# initial map object centered on Abuja
Map5 = geemap.Map(center=[center_lat,center_lon], zoom=zoomlevel)

# add our alternate basemap
Map5.add_basemap("SATELLITE")

# add our 1992 (and remember to create a mask and change opacity to 75%)
Map5.addLayer(dmsp92.mask(dmsp92), name='DMSP NTL 1992 masked', opacity=0.75)

Map5.addLayerControl()
Map5

## Visual inspection

Now you can look at nighttime layer and compare it to our satellite view basemap.

Take a look around. Interact with the map you just created:
- Toggle the satellte basemap off to compare nighttime lights to the road network as well as the satellite view. There should be an icon in the upper right corner that will  allow you to do this.
- Navigate to Abuja and zoom in.
- Can you see where the overlap of the nighttime lights are with the roads and "built up" areas?
- How well do they overlap?
- Are there any surprises?
- What about other parts of Nigeria?

## Add a layer for DMSP-OLS 2013

Now let's look at nighttime lights for 2013.

To do this, we can just add a new layer to our object.

Can you do this on your own?

In [ ]:
# your code here


<br><br><br><br><br><br><br><br><br><br><br><br>

Need some hints?

In [ ]:
# find the Image ID for DMSP-OLS 2013 and set it as a new variable (hint: the satellite's name is "F18")


# create the ee object


# initialize a map object, centered on Abuja


# name it "DMSP NTL 2013", create a mask, and give it an opacity of 75%.


Scroll on for more hints...

<br><br><br><br><br><br><br><br><br><br><br><br>

In [ ]:
# find the Image ID and set it as a new variable (hint: the satellite's name is "F18")
dmsp2013id = "NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F182013"

# create the ee object
dmsp2013 = ee.Image(dmsp2013id)

# initial map object centered on Abuja
Map6 = geemap.Map(center=[center_lat,center_lon], zoom=zoomlevel)

# name it "DMSP NTL 2013" and give it an opacity of 75%.
Map6.addLayer(dmsp2013.mask(dmsp2013), name='DMSP NTL 2013', opacity=0.75)

Map6.addLayerControl()
Map6

Map(center=[9.0, 7.4], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

Now that you have both years, you can toggle back and forth and compare the differences.

Do you see any major changes?

Zoom in for a closer look at Abuja: do you see the growth from 1992 to 2013?

## Create a split planel view

<div class="alert alert-warning">
Warning, this is based on `ipyleaflet` a Python library that does not play well with Google Colab, so this code will not work in the Google Colab environment but should on your local machine.</div>

We've added our 2013 layer to compare with 1992, but it's kind of annoying to toggle each layer. It's also hard to truly compare. If we create a split panel view with a slider, we can more easily see the difference.

There is a built-in method in `geemap` for this, which makes it simple to do.

We've already created our 1992 and 2013 DMSP image objects and saved those as variables, so no need to re-create. We just need to generate a tile layer with each. But remember to mask them and let's again set opacity to 75%:

In [ ]:
# generate tile layers from the ee image objects, masking and changing opacity to 75%
dmsp92_tile = geemap.ee_tile_layer(dmsp92.mask(dmsp92), {}, 'DMSP NTL 1992', opacity=0.75)
dmsp2013_tile = geemap.ee_tile_layer(dmsp2013.mask(dmsp2013), {}, 'DMSP NTL 2013', opacity=0.75)

Initialize as before, but now you can also just alter the initial object we created, which is what we'll do. We can call the object's `.split_map()` method and set the left and panels with our 1992 and 2013 tile layers.

In [ ]:
# initial map object centered on Abuja
Map7 = geemap.Map(center=[center_lat,center_lon], zoom=zoomlevel)

# use .split_map function to create split panels
Map7.split_map(left_layer=dmsp92_tile, right_layer=dmsp2013_tile)


Map7.addLayerControl()
Map7

Map(center=[9.0, 7.4], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

# Tutorial 2

### Get DMSP-OLS Image Collection and review meta-data and description

Recall the image collection for DMSP-OLS is located at: `NOAA/DMSP-OLS/NIGHTTIME_LIGHTS`

In [ ]:
dmsp = ee.ImageCollection("NOAA/DMSP-OLS/NIGHTTIME_LIGHTS")

**What is the total number of images in this collection?**
(It spans 1992-2013, but some years contain multiple images due to satellite overlap.)

In the Google Earth Engine Editor, we call the `.size()` method on our collection.

With `geemap` we can do the same thing, however, `.size()` will produce a size "object", so we have to add the extra step of using the `.getInfo()` method so it prints out to our notebook.

In the GEE editor (in JavaScript):
```
print(dmsp.size());
```

In our Python notebook:

In [ ]:
print(dmsp.size().getInfo())

35


**TIP:** We can use Python's functional string method by adding `f` and brackets to dynamically print our collection size in a sentence:

In [ ]:
print(f"There are {dmsp.size().getInfo()} images in this collection.")

There are 35 images in this collection.


**What is the date range of our collection?**

GEE has a set of methods called "Reducers," which do a range of functions, such as get the sum or avg value of a collection. They are quite handy. The function `Reducer.minMax()` can be used to get a date range.


In the GEE editor (in JavaScript):
```
var imgrange = dmsp.reduceColumns(ee.Reducer.minMax(), ["system:time_start"]);
var start = ee.Date(imgrange.get('min'));
var end = ee.Date(imgrange.get('max'));
print('Date range: ', start, end);
```

Let's try this in Python...

In [ ]:
imgrange = dmsp.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
start = ee.Date(imgrange.get('min')).getInfo()
end = ee.Date(imgrange.get('max')).getInfo()
print(f"Date range: {start, end}")

Date range: ({'type': 'Date', 'value': 694224000000}, {'type': 'Date', 'value': 1356998400000})


Huh? One thing the GEE editor does is unpacks the JSON output (like a Python dictionary).

We can do that here by accessing the "value" key from our `.getInfo()` object. Let's try it again:

In [ ]:
imgrange = dmsp.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
start = ee.Date(imgrange.get('min')).getInfo()['value']
end = ee.Date(imgrange.get('max')).getInfo()['value']
print(f"Date range: {start, end}")

Date range: (694224000000, 1356998400000)


OK...these numbers still don't make any sense. What's happening?

### Date conversions...

Tracking time in a world with many timezones is incredibly complicated. And you can imagine the nightmare scenario if applications (like airplane navigation systems) relied on arbitrary or inconsistent methods of tracking time. Therefore, software applications (including those that preprocessed our DMSP-OLS files as well as GEE) use what's called `Unix time`, which is what we see here.


<div class="alert alert-info">
    We'll spare you the <a href="https://en.wikipedia.org/wiki/Unix_time">hairy details</a>, but basically, <b>Unix time a.k.a. Epoch time</b> refers to the time that has elapsed (not counting leap seconds) since the begining of the "Unix epoch," which started at 00 hrs 00 minutes 00 seconds on 1 January, 1970 in Coordinated Universal Time (UTC). There are two important conventions you should assume unless stated otherwise: 1) this number is measured in milliseconds (a positive integer after Jan 1, 1970, negative before) and 2) it is in UTC time (equivalent to Greenwich Mean Time).
</div>

It's an extra step to convert this to a date that we humans understand in Python, but understanding timestamps is critical when working with temporal data -- especially data produced by satelliltes orbiting the planet, so hopefully this helps you appreciate working with data that spans time and space!

To convert these Unix time values to readable dates, we'll:
1. Divide by 1000 to convert to seconds and
2. Use Python's handy `datetime` library to convert to datetime object using the `utcfromtimestamp` method (recall the time is in UTC!)
3. Convert this datetime object to a string that we can read with the pattern: Year (%Y) - Month (%m) - Day (%d) Hour (%H): Minute (%M): Second (%S)

In [ ]:
imgrange = dmsp.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
start = ee.Date(imgrange.get('min')).getInfo()['value']
end = ee.Date(imgrange.get('max')).getInfo()['value']

# convert date
from datetime import datetime
start = datetime.utcfromtimestamp(start/1000).strftime('%Y-%m-%d %H:%M:%S')
end = datetime.utcfromtimestamp(end/1000).strftime('%Y-%m-%d %H:%M:%S')
print(f"Date range: {start, end}")

Date range: ('1992-01-01 00:00:00', '2013-01-01 00:00:00')


OK! That makes sense.

So to recap...

In the GEE editor (in JavaScript):
```
var imgrange = dmsp.reduceColumns(ee.Reducer.minMax(), ["system:time_start"]);
var start = ee.Date(imgrange.get('min'));
var end = ee.Date(imgrange.get('max'));
print('Date range: ', start, end);
```

In Python:
```
from datetime import datetime

imgrange = dmsp.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
start = ee.Date(imgrange.get('min')).getInfo()['value']
end = ee.Date(imgrange.get('max')).getInfo()['value']

start = datetime.utcfromtimestamp(start/1000).strftime('%Y-%m-%d %H:%M:%S')
end = datetime.utcfromtimestamp(end/1000).strftime('%Y-%m-%d %H:%M:%S')
print(f"Date range: {start, end}")
```

If this is a method you'll want to run more than once, you should create a function from these lines of code that only needs the image collection as a parameter.

In [ ]:
def get_date_range(img_collection):
    imgrange = img_collection.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
    start = ee.Date(imgrange.get('min')).getInfo()['value']
    end = ee.Date(imgrange.get('max')).getInfo()['value']

    start = datetime.utcfromtimestamp(start/1000).strftime('%Y-%m-%d %H:%M:%S')
    end = datetime.utcfromtimestamp(end/1000).strftime('%Y-%m-%d %H:%M:%S')
    print(f"Date range: {start, end}")

### Get DMSP-OLS annual composite for 1996

**Recall from** {doc}`mod1_2_introduction_to_nighttime_light_data`:<br><a href="https://ngdc.noaa.gov/eog/dmsp/downloadV4composites.html" class="alert-link">NOAA`s
National Center for Environmental Information</a> has the reference table of DMSP satellites:


**DMSP-OLS satellites by year**

|      | F10     | F12     | F14     | F15     | F16     | F18     |
|------|---------|---------|---------|---------|---------|---------|
| 1992 | F101992 |         |         |         |         |         |
| 1993 | F101993 |         |         |         |         |         |
| 1994 | F101994 | F121994 |         |         |         |         |
| 1995 |         | F121995 |         |         |         |         |
| 1996 |         | F121996 |         |         |         |         |
| 1997 |         | F121997 | F141997 |         |         |         |
| 1998 |         | F121998 | F141998 |         |         |         |
| 1999 |         | F121999 | F141999 |         |         |         |
| 2000 |         |         | F142000 | F152000 |         |         |
| 2001 |         |         | F142001 | F152001 |         |         |
| 2002 |         |         | F142002 | F152002 |         |         |
| 2003 |         |         | F142003 | F152003 |         |         |
| 2004 |         |         |         | F152004 | F162004 |         |
| 2005 |         |         |         | F152005 | F162005 |         |
| 2006 |         |         |         | F152006 | F162006 |         |
| 2007 |         |         |         | F152007 | F162007 |         |
| 2008 |         |         |         |         | F162008 |         |
| 2009 |         |         |         |         | F162009 |         |
| 2010 |         |         |         |         |         | F182010 |
| 2011 |         |         |         |         |         | F182011 |
| 2012 |         |         |         |         |         | F182012 |
| 2013 |         |         |         |         |         | F182013 |

In 1996, we have but one option to choose: `F121996`

## Add this composite to our map

Lets add this image as a layer to our map.

Let's also apply the mask to areas with no data and adjust the opacity to 75%.

Recall how to do this from {doc}`mod2_6_practical_exercise-image_visualization`

In [ ]:
dmsp1996 = ee.Image("NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F101992")

# initialize our map
map2 = geemap.Map(center=[center_lat,center_lon], zoom=zoomlevel)
map2.add_basemap('SATELLITE')

map2.addLayer(dmsp1996.mask(dmsp1996), {}, "DMSP-OLS 1996", opacity=0.75)

map2.addLayerControl()
map2

Map(center=[9.0, 7.4], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

## Add a 2nd annual composite from another year (2010) and create a slider panel to view and compare both.

#### Warning, this is based on `ipyleaflet` a Python library that does not play well with Google Colab, so this code will not work in the Google Colab environment but should on your local machine.

In [ ]:
# get the satellite name from the reference table
dmsp2010 = ee.Image("NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F182010")

map2.addLayer(dmsp2010.mask(dmsp2010), {}, "DMSP-OLS 2010", opacity=0.75)

Recall how to create a split panel slider from {doc}`mod2_6_practical_exercise-image_visualization`:

<div class="alert alert-warning">
Warning, this is based on `ipyleaflet` a Python library that does not play well with Google Colab, so the split panel display will not work in the Google Colab environment but should on your local machine.</div>

In [ ]:
# initialize our map
map3 = geemap.Map(center=[center_lat,center_lon], zoom=zoomlevel)
map3.add_basemap('SATELLITE')

# generate tile layers
dmsp1996_tile = geemap.ee_tile_layer(dmsp1996.mask(dmsp1996), {}, 'DMSP-OLS 1996', opacity=0.75)
dmsp2010_tile = geemap.ee_tile_layer(dmsp2010.mask(dmsp2010), {}, 'DMSP-OLS 2010', opacity=0.75)

# create split map
map3.split_map(left_layer=dmsp1996_tile, right_layer=dmsp2010_tile)
map3

Map(center=[9.0, 7.4], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

**Great!** Now you've found and visualized an annual composite as well as learned a few helpful ways to access meta-data. This slider is cool as a visualization, but as noted in the first tutorial, the DMSP-OLS satellites do not have on-board calibration. As a result, the change from satellite to satellite (as is the case when comparing 1996 to 2010) might include sensor variations that do not represent actual observed changes in light.

This is why you should be very careful when comparing or analyzing change in DMSP-OLS series directly.

The way to address this is by adjusting our DMSP-OLS annual composites through a process we'll call "intercalibration" and this is the subject of a later tutorial.

## References:
```{bibliography} ../references.bib
:filter: docname in docnames
```

# Tutorial 3

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/worldbank/OpenNightLights/blob/master/onl/tutorials/mod3_2_image_clipping_with_VIIRS.ipynb)

# Image clipping with VIIRS-DNB (5 min)

Satellite data, which often comes structured as a GeoTIFF if you recall from  {doc}`mod2_1_data_overview`, can cover large areas geospatially. It's not always required to work with the entire file, in fact, it's often preferred to work only with a smaller Area Of Interest (AOI).

In this tutorial, we're going to show how you can clip a particular satellite raster file to a specific AOI, including a geometry from a geopolitical boundary. We'll also apply this clipping to an entire ImageCollection.

For this exercise, we'll work with the VIIRS-DNB data.

**Our tasks in this exercise:**
1. Get a monthly VIIRS-DNB composite and clip to a Geometry (a buffer around a point)
2. Clip the image to the geometry of the state of California.
3. Clip all the images in the VIIRS-DNB stray-light corrected image collection to the state of California

## Get and clip a VIIRS-DNB monthly composite

For this exercise, we'll look at the VIIRS-DNB monthly composite for December 2019.

#### Initialize map, get image and add as layer

In [7]:
try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

# get December image, we're using the "avg_rad" band
viirs2019_12 = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate("2019-12-01","2019-12-31").select('avg_rad').median()


# initialize our map
map1 = geemap.Map()
map1.add_basemap('SATELLITE')
map1.addLayer(viirs2019_12, {}, "VIIRS-DNB Dec 2019")

map1.addLayerControl()
map1

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Clip our image to an area around Los Angeles, CA

We'll create a circular AOI using the `ee.Geometry.Point` method at lon: -118.2541, lat: 34.0469 (downtown LA) and then create a 200,000 meter (200 km) buffer around that.

In [ ]:
lat = 34.0469
lon = -118.2541
aoi = ee.Geometry.Point([lon, lat]).buffer(200000);

Now, let's clip our VIIRS image to our AOI and add that as a layer by passing our `aoi` object to the `.clip()` function, like this:

In [ ]:
viirs2019_12_clipped = viirs2019_12.clip(aoi)

map2 = geemap.Map(center=[lat, lon],zoom=7)
map2.add_basemap('SATELLITE')
map2.addLayer(viirs2019_12_clipped, {}, "VIIRS-DNB- Greater LA Dec 2019")
map2.addLayerControl()
map2

Map(center=[34.0469, -118.2541], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(c…

Now we can focus our attention (for analysis or visualization) on just the area around greater LA.

## Clip an image to a geopolitical boundary: State of California

We arbitrarily chose a lat/lon point and buffer to clip our image around, but in geospatial analysis, we're often called to focus on particular geopolitical boundaries.

One example might be to use the shapefile of a country or sub-national boundary to clip our image.

#### Importing shapefiles with `geemap`

Importing shapefiles is very easy to do with geemap. It's as simple as:

```
ee_object = geemap.shp_to_ee(<pathtomyshapefile>)
```

...which you can now add to your map object:

```
Map.addLayer(ee_object, {}, 'Layer name')
```

So if you had a shapefile, say for the state of California, you could import that as just shown, get the geomtry of the shapefile (i.e. by selecting the `geometry` field) and then clip your image to that geometry, just like we did with `aoi` earlier).

Conveniently, Google Earth Engine actually has geometries for US States as a native FeatureCollection so we dont need to get those. These are based on the US Census TIGER files from 2016 and are located in GEE at `TIGER/2016/States`

We'll retrieve the geometry for the state of California:

In [ ]:
aoi_CA = ee.FeatureCollection('TIGER/2016/States').filter(ee.Filter.eq('NAME', 'California'))

Now, just as with our previous AOI, we can clip our VIIRS Dec 2019 composite image to the entire state of California.

In [ ]:
viirs2019_12_ca = viirs2019_12.clip(aoi_CA)

map3 = geemap.Map(center=[lat, lon],zoom=5)
map3.add_basemap('SATELLITE')
map3.addLayer(viirs2019_12_ca, {}, "VIIRS-DNB- California Dec 2019")
map3.addLayerControl()
map3

Map(center=[51.763717735921325, 19.46006792642923], controls=(WidgetControl(options=['position', 'transparent_…

## Clip an entire ImageCollection to the State of California

So far, we've just clipped the December 2019 VIIRS-DNB composite, but we can easily apply this geometry clipping to an entire Image Collection. This would help if we wanted to create time series analysis of our focus area.

#### ImageCollection `map`

Here's it's necessary to learn about the `.map()` function of ImageCollections, which as the name suggests, maps a given function each image in the collection, with some constraints as <a href="https://developers.google.com/earth-engine/guides/ic_mapping">you can read here.</a>

Previously, we passed our geometry (e.g. `aoi_CA`) to the `.clip()` function to clip a single image. We can map a function that does this to our entire ImageCollection in Python by first defining a simple function that clips a single image to our specific AOI (California).

In [ ]:
# get the entire VIIRS-DNB selection -- still only selecting the "avg_rad" band
viirsDNB = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").select('avg_rad')

# here's a function that uses the CA aoi to clip an image
def clip_func(x):
    return x.clip(aoi_CA)

# for simple functions in Python, we can also use the "lambda" convention
# this is identical to what we just defined above
# and is nice and compact for vectorizing functions on arrays and images, etc.
clip_func = lambda x: x.clip(aoi_CA)

# in fact, by using lambda, we dont need to define this "clip_func" function at all
# we can write it directly in our "map" function, like this:
viirs_CA = viirsDNB.map(lambda x: x.clip(aoi_CA))

Now we can query this image collection for any date or series of dates to visualize layers.

Let's look at June of 2015 and June 2020, for instance:

In [ ]:
viirs_CA_2015_06 = viirs_CA.filterDate('2015-06-01','2015-06-30').median()
viirs_CA_2019_06 = viirs_CA.filterDate('2019-06-01','2019-06-30').median()

map4 = geemap.Map(center=[lat, lon],zoom=5)
map4.add_basemap('SATELLITE')
map4.addLayer(viirs_CA_2015_06, {}, "VIIRS-DNB- California June 2015")
map4.addLayer(viirs_CA_2019_06, {}, "VIIRS-DNB- California June 2019")
map4.addLayerControl()
map4

Map(center=[34.0469, -118.2541], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(c…

#### Annual composite

We could also get an annual composite of California for all of 2019, by reducing our image collection (through the `.median()` function.

In [ ]:
viirs_CA_2019 = viirs_CA.filterDate('2019-01-01','2019-12-31').median()

map5 = geemap.Map(center=[lat, lon],zoom=5)
map5.add_basemap('SATELLITE')
map5.addLayer(viirs_CA_2019, {}, "VIIRS-DNB- California 2019 (median)")
map5.addLayerControl()
map5

Map(center=[34.0469, -118.2541], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(c…

You can also get the the difference (i.e. change) in nighttime lights by subtracting images -- and much more. We'll cover operations like that in following exercises, but now you know how to clip an image to a specific geometry.

# Własne - Polska

In [8]:
lat = 51.763717735921325
lon = 19.46006792642923
aoi = ee.Geometry.Point([lon, lat]).buffer(200000);

In [26]:
viirs2019_12_clipped = viirs2019_12.clip(aoi)

map2 = geemap.Map(center=[lat, lon],zoom=7)
map2.add_basemap('ROADMAP')
map2.addLayer(viirs2019_12_clipped, {}, "VIIRS-DNB- California June 2015", opacity=0.9)
map2.addLayerControl()
map2

Google Maps has been already added before.


Map(center=[51.763717735921325, 19.46006792642923], controls=(WidgetControl(options=['position', 'transparent_…

In [29]:
aoi_CA = ee.FeatureCollection('FAO/GAUL/2015/level0').filter(ee.Filter.eq('ADM0_NAME', 'Poland'))
viirs2019_12 = ee.Image("NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F182010")#.filterDate("2019-12-01","2019-12-31").select('avg_rad').median()
 #FeatureCollection
viirs2019_12_ca = viirs2019_12.clip(aoi_CA)

map3 = geemap.Map(center=[lat, lon],zoom=5)
map3.add_basemap('ROADMAP')
map3.addLayer(viirs2019_12_ca, {}, "DMSP-OLS 1996", opacity=0.9)
#NOAA/DMSP-OLS/NIGHTTIME_LIGHTS   ; VIIRS-DNB- California Dec 2019
map3.addLayerControl()
map3

Google Maps has been already added before.


Map(center=[51.763717735921325, 19.46006792642923], controls=(WidgetControl(options=['position', 'transparent_…

In [30]:
# initialize our map
dmsp2010 = ee.Image("NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F182010")
dmsp2010_c = dmsp2010.clip(aoi_CA)
dmsp1996 = ee.Image("NOAA/DMSP-OLS/NIGHTTIME_LIGHTS/F101992")
dmsp1996_c = dmsp1996.clip(aoi_CA)

map3 = geemap.Map(center=[lat,lon], zoom=6)
map3.add_basemap('ROADMAP')

# generate tile layers
dmsp1996_tile = geemap.ee_tile_layer(dmsp1996_c, {}, 'DMSP-OLS 1996', opacity=0.75)
dmsp2010_tile = geemap.ee_tile_layer(dmsp2010_c, {}, 'DMSP-OLS 2010', opacity=0.75)

# create split map
map3.split_map(left_layer=dmsp1996_tile, right_layer=dmsp2010_tile)
map3

Google Maps has been already added before.


Map(center=[51.763717735921325, 19.46006792642923], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [51]:
# initialize our map
viirs2000_12 = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate("2014-05-01","2014-05-31").select('avg_rad').median()
viirs2000_12_c = viirs2000_12.clip(aoi_CA)

viirs2019_12 = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate("2018-12-01","2018-12-31").select('avg_rad').median()
viirs2019_12_cc = viirs2019_12.clip(aoi_CA)



map3 = geemap.Map(center=[lat,lon], zoom=6)
map3.add_basemap('ROADMAP')

# generate tile layers
dmsp2010_tile = geemap.ee_tile_layer(viirs2000_12_c, {}, 'VIIRS-DNB- California June 2015', opacity=0.8)
dmsp1996_tile = geemap.ee_tile_layer(viirs2019_12_cc, {}, 'VIIRS-DNB- California June 2015', opacity=0.8)


# create split map
map3.split_map(left_layer=dmsp1996_tile, right_layer=dmsp2010_tile)
map3

Google Maps has been already added before.


Map(center=[51.763717735921325, 19.46006792642923], controls=(ZoomControl(options=['position', 'zoom_in_text',…